In [1]:
def inverse_normalize(normalized, min_val, max_val):
    return ((normalized + 1) / 2) * (max_val - min_val) + min_val

In [2]:
from torch.utils import data
from stresnet_dataloader import St_ResNet_dataloader


train_dataset = St_ResNet_dataloader(
        name=["long_term.npy", "short_term.npy", "ingredients.npy", "externel.npy", "future.npy", "label.npy"],
        root="/Users/gunneo/Documents/4_2/Graduation_Thesis/Datasets/Beijing_House_Price_Dataset/St-ResNet/train",
    )
train_dataloader = data.DataLoader(train_dataset, batch_size=36, shuffle=True)
global_min = train_dataset.global_min[-1]
global_max = train_dataset.global_max[-1]
img_h = train_dataloader.dataset[0][0].shape[1]
img_w = train_dataloader.dataset[0][0].shape[2]
long_term_in_channels = train_dataloader.dataset[0][0].shape[0]
short_term_in_channels = train_dataloader.dataset[0][1].shape[0]
cur_ingred_dim = train_dataloader.dataset[0][2].shape[0]

print(img_h, img_w, long_term_in_channels, short_term_in_channels, cur_ingred_dim)
print(global_min, global_max)

30 30 2 12 4
0.0 900.4454545454545


In [3]:
from stresnet import STResNet

train_model = STResNet(
    len_long=2,
    len_short=12,
    len_ingred=4,
    external_dim=6,
    map_height=30,
    map_width=30,
    residual_unit=4,
)

In [4]:
import os
import torch
import torch.nn as nn
from torch.optim import Adam, SGD, RMSprop, Adagrad

def mse_loss(pred, target):
    mse = nn.MSELoss()
    return mse(pred, target)

def train(
        model,
        train_dataloader,
        w_path,
        num_epochs=50,
        optimizer_type='adam',
        lr=5e-4,
        patience=5,
        save_interval=25,
        loss_threshold=1e-4
        ):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
        
    optimizers = {
        'adam': Adam(model.parameters(), lr=lr),
        'sgd': SGD(model.parameters(), lr=lr, momentum=0.9),
        'rmsprop': RMSprop(model.parameters(), lr=lr),
        'adagrad': Adagrad(model.parameters(), lr=lr)
    }
    if optimizer_type not in optimizers:
        raise ValueError("Unsupported optimizer type")
    optimizer = optimizers[optimizer_type]

    best_model_path = os.path.join(w_path, "best_model.pth")
    best_loss = float('inf')
    no_improve = 0

    for epoch in range(num_epochs):
        model.train()
        epoch_train_loss = 0.0
        
        for batch in train_dataloader:
            labels = batch[-1].to(device)
            optimizer.zero_grad()
            outputs = model(batch[0], batch[1], batch[2], batch[3])
            loss = mse_loss(outputs, labels)
            loss.backward()
            optimizer.step()
            
            epoch_train_loss += loss.item() * labels.size(0)
        
        epoch_train_loss /= len(train_dataloader.dataset)
        print(f"Epoch [{epoch+1}/{num_epochs}] Train Loss: {epoch_train_loss:.4f}")

        if abs(epoch_train_loss - best_loss) >= loss_threshold:
            best_loss = epoch_train_loss
            torch.save(model.state_dict(), best_model_path)
            print(f"Best model saved to {best_model_path} with loss {best_loss:.4f}")
            no_improve = 0
        else:
            no_improve += 1
            print(f"No improvement for {no_improve}/{patience} epochs")
            if no_improve > patience:
                print(f"Early stopping at epoch {epoch+1}")
                break

        if save_interval > 0 and (epoch + 1) % save_interval == 0:
            checkpoint_path = os.path.join(w_path, f"checkpoint_{epoch+1}.pth")
            torch.save(model.state_dict(), checkpoint_path)
            print(f"Checkpoint saved at {checkpoint_path}")

    print("Training complete.")

In [5]:
train(
    train_model,
    train_dataloader,
    w_path="/Users/gunneo/Documents/4_2/Graduation_Thesis/Datasets/Beijing_House_Price_Dataset/St-ResNet/train_weights/",
    num_epochs=1000,
    optimizer_type='sgd',
    lr=1e-3,
    patience=20,
    save_interval=25,
    loss_threshold=5e-4
)

Epoch [1/1000] Train Loss: 1.7663
Best model saved to /Users/gunneo/Documents/4_2/Graduation_Thesis/Datasets/Beijing_House_Price_Dataset/St-ResNet/train_weights/best_model.pth with loss 1.7663
Epoch [2/1000] Train Loss: 1.7444
Best model saved to /Users/gunneo/Documents/4_2/Graduation_Thesis/Datasets/Beijing_House_Price_Dataset/St-ResNet/train_weights/best_model.pth with loss 1.7444
Epoch [3/1000] Train Loss: 1.7141
Best model saved to /Users/gunneo/Documents/4_2/Graduation_Thesis/Datasets/Beijing_House_Price_Dataset/St-ResNet/train_weights/best_model.pth with loss 1.7141
Epoch [4/1000] Train Loss: 1.6826
Best model saved to /Users/gunneo/Documents/4_2/Graduation_Thesis/Datasets/Beijing_House_Price_Dataset/St-ResNet/train_weights/best_model.pth with loss 1.6826
Epoch [5/1000] Train Loss: 1.6476
Best model saved to /Users/gunneo/Documents/4_2/Graduation_Thesis/Datasets/Beijing_House_Price_Dataset/St-ResNet/train_weights/best_model.pth with loss 1.6476
Epoch [6/1000] Train Loss: 1.6110
B

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
criterion = nn.MSELoss()
num_batches = 0
rmse_total = 0.0
with torch.no_grad():
    for batch in train_dataloader:
        labels = batch[-1].to(device)
        
        predictions = train_model(batch[0], batch[1], batch[2], batch[3])
        # convert the normalized prediction back to the original scale
        print(predictions[0][0][14])
        print(labels[0][0][14])
        mse = criterion(predictions, labels)
        rmse = torch.sqrt(mse)
        rmse_total += rmse.item()
        num_batches += 1

average_rmse = rmse_total / num_batches
print(f"Average RMSE: {average_rmse}")

tensor([-1.0000, -1.0000, -1.0000,  1.0000, -1.0000,  1.0000, -1.0000, -1.0000,
         1.0000, -1.0000,  1.0000, -1.0000,  1.0000, -1.0000, -1.0000, -1.0000,
        -1.0000, -1.0000,  0.0570, -1.0000, -1.0000, -0.9997, -1.0000, -1.0000,
        -0.5609, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000])
tensor([-0.5043, -0.4886, -1.0000, -0.5822, -0.5987, -0.1609, -0.6779, -1.0000,
         0.4214,  0.2620,  0.2470, -0.3159,  0.1424,  0.1360, -0.2504,  0.1291,
         0.0321, -0.0202, -0.2058, -0.0364,  0.0646, -0.2931, -0.3752, -0.1235,
        -0.5263, -1.0000, -0.6224, -0.4447, -0.3917, -1.0000])
Average RMSE: 0.546391487121582


In [15]:
test_model = STResNet(
    len_long=2,
    len_short=12,
    len_ingred=4,
    external_dim=6,
    map_height=30,
    map_width=30,
    residual_unit=4,
)

In [16]:
test_dataset = St_ResNet_dataloader(
        name=["long_term.npy", "short_term.npy", "ingredients.npy", "externel.npy", "future.npy", "label.npy"],
        root="/Users/gunneo/Documents/4_2/Graduation_Thesis/Datasets/Beijing_House_Price_Dataset/St-ResNet/test",
    )
test_dataloader = data.DataLoader(test_dataset, batch_size=12, shuffle=True)
test_global_min = test_dataset.global_min
test_global_max = test_dataset.global_max

In [17]:
test_model.load_state_dict(torch.load("/Users/gunneo/Documents/4_2/Graduation_Thesis/Datasets/Beijing_House_Price_Dataset/St-ResNet/train_weights/best_model.pth"))

/var/folders/wv/8hm9fpxs3tdgdkp2z_jy1mr40000gn/T/ipykernel_8365/50987390.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  test_model.load_state_dict(torch.load("/Users/gu

<All keys matched successfully>

In [18]:
def test(
    test_model,
    test_dataloader
):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    test_model = test_model.to(device)
    num_batches = 0
    rmse_total = 0.0
    with torch.no_grad():
        for batch in test_dataloader:
            labels = batch[-1].to(device)
            
            predictions = test_model(batch[0], batch[1], batch[2], batch[3])
            print(predictions[0][0][14])
            print(labels[0][0][14])
            rmse = torch.sqrt(mse_loss(predictions, labels))
            rmse_total += rmse.item()
            num_batches += 1

    average_rmse = rmse_total / num_batches
    print(f"Average RMSE: {average_rmse}")

In [19]:
test(test_model, test_dataloader)

tensor([-0.9948, -1.0000, -1.0000,  1.0000, -1.0000,  1.0000, -1.0000, -1.0000,
         1.0000, -1.0000,  1.0000,  1.0000,  1.0000, -1.0000, -1.0000, -1.0000,
        -1.0000,  1.0000,  1.0000,  1.0000, -1.0000, -1.0000, -0.9864, -0.9472,
        -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000])
tensor([-0.5894, -0.3720, -1.0000, -0.5195, -1.0000,  0.2337, -1.0000, -1.0000,
         0.1988, -0.0146,  0.1806, -0.0150,  0.2398,  0.0612, -0.5572, -1.0000,
         0.1289, -0.1843, -0.1384, -0.0016,  0.0295, -0.4614, -0.3094,  0.1091,
        -0.4264, -1.0000, -1.0000, -0.5681, -0.3154, -1.0000])
Average RMSE: 0.8094154596328735
